In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

Modeling

In [4]:
train_data = pd.read_csv("./data/train_data.csv")
sample_submission = pd.read_csv("./data/sample_submission.csv")
turb_location = pd.read_csv("./data/turb_location.CSV")

# ARMA

In [6]:
train_data.head()

,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv
0,1,1,00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,00:10,6.17,-3.99,30.73,41.80,25.92,1.0,1.0,1.0,-0.25,494.66
2,1,1,00:20,6.27,-2.18,30.60,41.63,20.91,1.0,1.0,1.0,-0.24,509.76
3,1,1,00:30,6.42,-0.73,30.52,41.52,20.91,1.0,1.0,1.0,-0.26,542.53
4,1,1,00:40,6.25,0.89,30.49,41.38,20.91,1.0,1.0,1.0,-0.23,509.36


In [5]:
sample_submission.head()

,index,TurbID,Day,Tmstamp,Wspd,Wdir,Etmp,Itmp,Ndir,Pab1,Pab2,Pab3,Prtv,Patv
0,0,1,201,00:00,0,0,0,0,0,0,0,0,0,9999
1,1,1,201,00:10,0,0,0,0,0,0,0,0,0,9999
2,2,1,201,00:20,0,0,0,0,0,0,0,0,0,9999
3,3,1,201,00:30,0,0,0,0,0,0,0,0,0,9999
4,4,1,201,00:40,0,0,0,0,0,0,0,0,0,9999


In [7]:
train_data = train_data.fillna(method = 'bfill')

col_lst = ['Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1', 'Pab2', 'Pab3', 'Prtv']

TurbID         0
Day            0
Tmstamp        0
Wspd       45587
Wdir       45587
Etmp       45587
Itmp       45587
Ndir       45587
Pab1       45587
Pab2       45587
Pab3       45587
Prtv       45587
Patv       45587
dtype: int64 

TurbID     0
Day        0
Tmstamp    0
Wspd       0
Wdir       0
Etmp       0
Itmp       0
Ndir       0
Pab1       0
Pab2       0
Pab3       0
Prtv       0
Patv       0
dtype: int64


In [9]:
sample_submission[sample_submission['TurbID'] == 1].shape[0]

288

In [ ]:
sample_submission_c = sample_submission.copy()

for col in col_lst:
    for i in range(1,135,1):
        y = train_data[(train_data['TurbID'] == i)][col]
        ARMAmodel = SARIMAX(y, order = (1, 0, 1))
        ARMAmodel = ARMAmodel.fit()

        y_pred = ARMAmodel.get_forecast(sample_submission_c[sample_submission_c['TurbID'] == i].shape[0])
        y_pred_df = y_pred.conf_int(alpha = 0.05) 
        y_pred_df["Predictions"] = ARMAmodel.predict(start = y_pred_df.index[0], end = y_pred_df.index[-1])

        y_pred_df.index = sample_submission_c[sample_submission_c['TurbID'] == i].index
        y_pred_out = y_pred_df["Predictions"]
        
        sample_submission_c[sample_submission_c['TurbID'] == i][col] = y_pred_out